# St. Clair County Land Use and Parcel Data Pipeline

In [1]:
!pip3 install Selenium
!pip3 install pandas
!pip3 install lxml
!pip3 install html5lib

In [2]:
import csv
import json
import math
import os
import pandas as pd
import pprint
import re
import requests
import time
import queue

In [3]:
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed, wait
from datetime import timedelta
from io import StringIO
from queue import Queue
from threading import Lock, RLock, Thread

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

## Extraction

In [4]:
search_pg = "https://stclairil.devnetwedge.com/"
cwd = os.getcwd()

### Download St. Clair Co. Property Tax Inquiry Selected Townships Parcel Listing 

In [ ]:
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--headless=new")
prefs = {"download.default_directory": f"{cwd}"}
options.add_experimental_option("prefs", prefs)

In [ ]:
driver = webdriver.Chrome(options)
driver.implicitly_wait(3)
driver.get(search_pg)

In [ ]:
# Click into Advanced Search Tab
advance_search_tab = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//a[@href='#advanced-search']"))
)
advance_search_tab.click()

In [ ]:
# Select Townships
township_select = Select(WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "advanced-search-townships"))
))
township_select.select_by_value("02")
township_select.select_by_value("11")
township_select.select_by_value("01")
township_select.select_by_value("06")

In [ ]:
# Check All Years Box and Search
all_years_chkbx = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "advanced-search-include-all-years"))
)
form = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//form"))
)
driver.execute_script(f"document.getElementById('advanced-search-include-all-years').click()")
all_years_chkbx.submit()

In [ ]:
# Export Results to CSV and Download
export_btn = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//a[@href='/Search/ExportClientsListToCSV']"))
)
export_btn.click()

In [ ]:
driver.quit()

### Scrape Parcel Information Tables

In [5]:
parcel_list_df = pd.read_csv("Exported_Search_Results.csv")

In [6]:
parcel_list_df.describe(include="all")

,Year,Property Account Number,Owner,Address,Type
count,75463.000000,75463,74741,75463,75463
unique,NaN,75463,38756,53628,1
top,NaN,00-00.0-000-000,"ST CLAIR COUNTY TRUSTEE,",,Parcel
freq,NaN,1,2384,693,75463
mean,2020.177743,NaN,NaN,NaN,NaN
std,4.797474,NaN,NaN,NaN,NaN
min,1989.000000,NaN,NaN,NaN,NaN
25%,2022.000000,NaN,NaN,NaN,NaN
50%,2022.000000,NaN,NaN,NaN,NaN
75%,2022.000000,NaN,NaN,NaN,NaN


In [7]:
parcel_list_df.head()

,Year,Property Account Number,Owner,Address,Type
0,2006,00-00.0-000-000,NaN,,Parcel
1,2022,01-11.0-200-008,"ILLINOIS AMERICAN WATER CO,",FRONT ST EAST SAINT LOUIS,Parcel
2,2022,01-11.0-200-010,"ILLINOIS AMERICAN WATER CO,",FRONT ST EAST SAINT LOUIS,Parcel
3,2022,01-11.0-200-017,"WIGGINS FERRY CO,",FRONT ST EAST SAINT LOUIS,Parcel
4,2022,01-11.0-200-021,"ILLINOIS AMERICAN WATER CO,",FRONT ST EAST SAINT LOUIS,Parcel


In [8]:
# Format PropertyAccountNumber to be Solely Numeric + 'X'
def only_numeric(str):
    return "".join(re.findall(r"[\dX]", str))

parcel_list_df['Property Account Number'] = parcel_list_df['Property Account Number'].apply(only_numeric)
parcel_list_df.head()

,Year,Property Account Number,Owner,Address,Type
0,2006,00000000000,NaN,,Parcel
1,2022,01110200008,"ILLINOIS AMERICAN WATER CO,",FRONT ST EAST SAINT LOUIS,Parcel
2,2022,01110200010,"ILLINOIS AMERICAN WATER CO,",FRONT ST EAST SAINT LOUIS,Parcel
3,2022,01110200017,"WIGGINS FERRY CO,",FRONT ST EAST SAINT LOUIS,Parcel
4,2022,01110200021,"ILLINOIS AMERICAN WATER CO,",FRONT ST EAST SAINT LOUIS,Parcel


In [9]:
with open("Exported_Search_Results.csv", newline="") as csvfile:
    reader = csv.reader(csvfile)
    parcel_list = list(reader)[2:]

url_pcs = [ (only_numeric(row[1]), row[0]) for row in parcel_list ]
display(len(url_pcs))
print(url_pcs[::1000])

75462

[('01110200008', '2022'), ('01130213056', '2022'), ('01130413004', '2022'), ('01230510002', '2022'), ('01240142050', '2022'), ('01240317097', '2020'), ('01240409068', '2010'), ('01250109013', '2022'), ('02010302006', '2006'), ('02040203082', '2022'), ('02070301041', '2008'), ('02070422014', '2021'), ('02080314039', '2017'), ('02090106005', '2022'), ('02090213105', '2022'), ('02090401028', '2022'), ('02100107041', '2022'), ('02100311004', '2022'), ('02120403025', '2022'), ('02150107032', '2022'), ('02160104095', '2022'), ('02160212004', '2022'), ('02160315001', '2022'), ('02170216068', '2017'), ('02170407036', '2022'), ('02180205089', '2022'), ('02180328062', '2022'), ('02190103036', '2022'), ('02190212011', '2022'), ('02190412015', '2003'), ('02200118052', '2022'), ('02200301001', '2022'), ('02210301016', '2022'), ('02220303021', '2022'), ('02220425012', '2022'), ('02250105025', '2022'), ('02260119001', '2022'), ('02260402023', '2022'), ('02300105024', '2009'), ('02350203031', '2022'),

In [10]:
def scrape_parcel_pg(listing_number, listing_year):
    parcel_url = f"{search_pg}parcel/view/{listing_number}/{listing_year}"
    parcel_pg = requests.get(parcel_url)
    parcel_pg = BeautifulSoup(parcel_pg.text, "html.parser")
    panel_divs = parcel_pg.find_all(class_="panel panel-info")
        
    tables_dict = {}
    
    for div in panel_divs:
        try:
            tbl_key = div.div.h3.text
            tbl = div.div.h3.parent.find_next_sibling().find("table").prettify()
            tables_dict[tbl_key] = pd.read_html(StringIO(tbl))[0]
        except:
            continue
    
    parcel_number = listing_number
    year = int(listing_year)

    try:
        # Property Information Table
        parcel_address = tables_dict['Property Information'][1][0].split("Site Address")[1].strip()
        sale_status = tables_dict["Property Information"][0][2].split("Sale Status")[1].strip()
        property_class = tables_dict["Property Information"][0][3].split("-")[0].split("Property Class")[1].strip()
        tax_status = tables_dict["Property Information"][2][3].split("Tax Status")[1].strip()
        net_taxable = tables_dict["Property Information"][0][4].split("Net Taxable Value")[1].strip()
        tax_rate = tables_dict["Property Information"][1][4].split("Tax Rate")[1].strip()
        total_tax = tables_dict["Property Information"][2][4].split("$")[1].strip()
        township = tables_dict["Property Information"][0][5].split("Township")[1].strip()
        acreage = tables_dict["Property Information"][1][5].split("Acres")[1].strip()
        
        # Assessments Table
        homesite_val = tables_dict["Assessments"].get("Homesite")[0]
        dwelling_val = tables_dict["Assessments"].get("Dwelling")[0]
        dept_rev_val = tables_dict["Assessments"].get("Total")[0]
    
        # Billing Table
        total_billed = tables_dict["Billing"].get("Totals")[4].strip("$")
        total_unpaid = tables_dict["Billing"].get("Totals")[6].strip("$")
    
        # Owner Information Table
        owner_name = tables_dict["Parcel Owner Information"].get("Name")[0]
        owner_address = tables_dict["Parcel Owner Information"].get("Address")[0]
    except Exception as err:
        time.sleep(1)
        return {
            "parcel": listing_number,
            "year": listing_year,
            "error": err
        }
    else:
        time.sleep(2)
        return {
            "parcel_number": parcel_number,
            "year": year,
            "parcel_address": parcel_address,
            "owner": owner_name,
            "owner_address": owner_address,
            "sale_status": sale_status,
            "property_class": property_class,
            "tax_status": tax_status,
            "net_taxable": net_taxable, 
            "tax_rate": tax_rate,
            "total_tax": total_tax,
            "township": township,
            "acreage": acreage,
            "homesite_val": homesite_val, 
            "dwelling_val": dwelling_val,
            "dept_rev_val": dept_rev_val,
            "total_billed": total_billed,
            "total_unpaid": total_unpaid
        }


In [15]:
def write_records2():
    
    function_start = time.perf_counter()
    max_threads = 1000
    processed_ct = 0
    records_missed = []
    parcel_records = []
    success_headers = [
        "parcel_number", "year", "parcel_address", "owner", "owner_address", 
        "sale_status", "property_class", "tax_status", "net_taxable", 
        "tax_rate", "total_tax", "township", "acreage", "homesite_val", 
        "dwelling_val", "dept_rev_val", "total_billed", "total_unpaid"
    ]
    fail_headers = ["parcel", "year", "error"]
    record_q = Queue() 
    csv_rlock = RLock()
    flush_lock = Lock()

    
    def write_CSV(filename, headers, data, r_lock):
        with r_lock:
            if os.path.exists(filename):
                with open(filename, mode="a", newline="") as records_file:
                    records_writer = csv.writer(records_file, dialect="excel")
                    records_writer.writerows(data)
                    data.clear()
            else:
                with open(filename, mode="a", newline="") as records_file:
                    records_writer = csv.writer(records_file, dialect="excel")
                    records_writer.writerow(headers)
                    records_writer.writerows(data)
                    data.clear()
    
    def process_records_q(rec_queue, batch_start_t, r_lock):
        parcel_info = rec_queue.get()
        if len(parcel_info) == 18:
            parcel_records.append(list(parcel_info.values()))
            if len(parcel_records) == 500:
                write_CSV("parcel_records.csv", success_headers, parcel_records, r_lock)
                # print(f"Writing batch(500) records took {timedelta(seconds=time.perf_counter()-batch_start_t)} seconds.")
                parcel_records.clear()
        if len(parcel_info) == 3:
            records_missed.append(list(parcel_info.values()))
            if len(records_missed) == 50:
                write_CSV("missed_records.csv", fail_headers, records_missed, r_lock)
                # print(f"Wrote 50 records missing data.")
                records_missed.clear()

    def flushing_writes(fxn_start_t, lock):
        write_CSV("parcel_records.csv", success_headers, parcel_records, lock)
        write_CSV("missed_records.csv", fail_headers, records_missed, lock)
        print(f"Writing parcel datums took: {timedelta(seconds=time.perf_counter()-fxn_start_t)} seconds.")
        records_df = pd.read_csv("parcel_records.csv")
        fail_df = pd.read_csv("missed_records.csv")
        display(records_df.describe(include="all"))
        display(fail_df.describe(include="all"))

    def q_flow():
        print("Running queue")
        while True:
            while record_q.empty() == False:
                process_records_q(record_q, batch_write_start, csv_rlock)
            else:
                time.sleep(3)
                continue
        
    q_thread = Thread(target=q_flow)
    # q_thread.start()

    url_list_length = len(url_pcs)
    decimals = [ i/10 for i in range(8, 11, 1) ] #CORRECT decimals = [ i/10 for i in range(1, 11, 1) ]
    split_length = [ url_list_length*decimal for decimal in decimals ]

    with ThreadPoolExecutor(max_workers=max_threads) as p1:
        begin_at = 60369 #CORRECT 0
        for split in split_length:
            print("Proceeding...")
            # q_thread.start()
            # batch_write_start = time.perf_counter()
            futures = [ p1.submit(scrape_parcel_pg, row[0], row[1]) for row in url_pcs[begin_at:math.floor(split)] ]
            if q_thread.is_alive() == False:
                q_thread.start()
            elif q_thread.is_alive() == True:
                pass
            batch_write_start = time.perf_counter()
            for future in as_completed(futures):
                record_q.put(future.result())
                processed_ct += 1
                # time.sleep(0.1)
                if processed_ct % 500 == 0:
                    print(f"Have processed {processed_ct} records in {timedelta(seconds=time.perf_counter()-batch_write_start)} seconds.")
                    batch_write_start = time.perf_counter()
                # if processed_ct % 1000 == 0:
            flushing_writes(function_start, flush_lock)
            begin_at = math.floor(split)
            time.sleep(30)
            print(f"Proceeding to next batch...setting at index {begin_at}..")
    print("Finished.")
    exit()
    
                


In [16]:
write_records2()

Proceeding...
Running queue
Writing parcel datums took: 0:00:00.001917 seconds.


,parcel_number,year,parcel_address,owner,owner_address,sale_status,property_class,tax_status,net_taxable,tax_rate,total_tax,township,acreage,homesite_val,dwelling_val,dept_rev_val,total_billed,total_unpaid
count,5.360900e+04,53609.000000,53608,53609,53609,4680,53609.000000,53609,53609,53609.000000,53609,53609,53609.000000,5.360900e+04,5.360900e+04,5.360900e+04,53609,53609
unique,NaN,NaN,40501,27518,27227,8,NaN,4,22149,NaN,25727,4,NaN,NaN,NaN,NaN,28402,26
top,NaN,NaN,"N 64TH ST EAST SAINT LOUIS, IL 62204",ST CLAIR COUNTY TRUSTEE,"10 PUBLIC SQ BELLEVILLE, IL, 62220",TRUSTSUB,NaN,Taxable,0,NaN,0.00,EAST ST LOUIS,NaN,NaN,NaN,NaN,0.00,0.00
freq,NaN,NaN,94,2415,3340,1900,NaN,50232,17497,NaN,17497,24722,NaN,NaN,NaN,NaN,17523,53584
mean,3.232086e+09,2020.972467,NaN,NaN,NaN,NaN,89.724169,NaN,NaN,9.929690,NaN,NaN,0.985539,4.990651e+03,1.981188e+04,2.490997e+04,NaN,NaN
std,2.024434e+09,3.462527,NaN,NaN,NaN,NaN,521.756302,NaN,NaN,5.215172,NaN,NaN,9.345429,3.459423e+04,6.930110e+04,8.779975e+04,NaN,NaN
min,1.110200e+09,2004.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN
25%,2.090414e+09,2022.000000,NaN,NaN,NaN,NaN,30.000000,NaN,NaN,7.793700,NaN,NaN,0.070000,3.960000e+02,0.000000e+00,4.840000e+02,NaN,NaN
50%,2.200216e+09,2022.000000,NaN,NaN,NaN,NaN,40.000000,NaN,NaN,9.993400,NaN,NaN,0.150000,1.089000e+03,2.022000e+03,5.633000e+03,NaN,NaN
75%,4.210300e+09,2022.000000,NaN,NaN,NaN,NaN,40.000000,NaN,NaN,13.834600,NaN,NaN,0.270000,4.715000e+03,2.510600e+04,3.060500e+04,NaN,NaN


,parcel,year,error
count,6758,6758.000000,6758
unique,6554,NaN,5
top,02190319010,NaN,'Billing'
freq,2,NaN,5864
mean,NaN,2012.360314,NaN
std,NaN,7.808171,NaN
min,NaN,1989.000000,NaN
25%,NaN,2005.000000,NaN
50%,NaN,2012.000000,NaN
75%,NaN,2020.000000,NaN


Proceeding to next batch...setting at index 60369..
Proceeding...
Have processed 500 records in 0:01:52.299991 seconds.
Have processed 1000 records in 0:00:41.423185 seconds.
Have processed 1500 records in 0:01:07.013745 seconds.
Have processed 2000 records in 0:00:44.278990 seconds.
Have processed 2500 records in 0:01:03.919594 seconds.
Have processed 3000 records in 0:00:48.837463 seconds.
Have processed 3500 records in 0:00:57.197484 seconds.
Have processed 4000 records in 0:00:47.389240 seconds.
Have processed 4500 records in 0:00:57.136186 seconds.
Have processed 5000 records in 0:00:45.350792 seconds.
Have processed 5500 records in 0:00:57.389257 seconds.
Have processed 6000 records in 0:00:45.646111 seconds.
Have processed 6500 records in 0:00:49.343608 seconds.
Have processed 7000 records in 0:00:18.994959 seconds.
Have processed 7500 records in 0:00:05.418364 seconds.
Writing parcel datums took: 0:13:37.552374 seconds.


,parcel_number,year,parcel_address,owner,owner_address,sale_status,property_class,tax_status,net_taxable,tax_rate,total_tax,township,acreage,homesite_val,dwelling_val,dept_rev_val,total_billed,total_unpaid
count,6.058100e+04,60581.000000,60580,60581,60581,5037,60581.000000,60581,60581,60581.000000,60581,60581,60581.000000,6.058100e+04,6.058100e+04,6.058100e+04,60581,60581
unique,NaN,NaN,46089,31757,31391,8,NaN,4,25615,NaN,30164,4,NaN,NaN,NaN,NaN,33127,26
top,NaN,NaN,"N 64TH ST EAST SAINT LOUIS, IL 62204",ST CLAIR COUNTY TRUSTEE,"10 PUBLIC SQ BELLEVILLE, IL, 62220",TRUSTSUB,NaN,Taxable,0,NaN,0.00,EAST ST LOUIS,NaN,NaN,NaN,NaN,0.00,0.00
freq,NaN,NaN,94,2424,3367,1956,NaN,56605,18711,NaN,18711,24722,NaN,NaN,NaN,NaN,18737,60556
mean,3.805280e+09,2021.017547,NaN,NaN,NaN,NaN,87.227893,NaN,NaN,9.743217,NaN,NaN,0.923139,5.311854e+03,2.058887e+04,2.600129e+04,NaN,NaN
std,2.480529e+09,3.383886,NaN,NaN,NaN,NaN,502.282637,NaN,NaN,5.069560,NaN,NaN,8.836341,3.503824e+04,7.189416e+04,9.070868e+04,NaN,NaN
min,1.110200e+09,2004.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN
25%,2.100319e+09,2022.000000,NaN,NaN,NaN,NaN,30.000000,NaN,NaN,7.793700,NaN,NaN,0.080000,4.270000e+02,0.000000e+00,5.530000e+02,NaN,NaN
50%,2.230311e+09,2022.000000,NaN,NaN,NaN,NaN,40.000000,NaN,NaN,9.571100,NaN,NaN,0.150000,1.340000e+03,5.031000e+03,7.339000e+03,NaN,NaN
75%,4.310411e+09,2022.000000,NaN,NaN,NaN,NaN,40.000000,NaN,NaN,13.834600,NaN,NaN,0.260000,4.975000e+03,2.627600e+04,3.170700e+04,NaN,NaN


...

Proceeding to next batch...setting at index 67915..
Proceeding...
Have processed 8000 records in 0:02:11.131557 seconds.
Have processed 8500 records in 0:00:38.584802 seconds.
Have processed 9000 records in 0:01:01.909552 seconds.
Have processed 9500 records in 0:00:42.240519 seconds.
Have processed 10000 records in 0:01:02.789620 seconds.
Have processed 10500 records in 0:00:47.577702 seconds.
Have processed 11000 records in 0:00:51.117704 seconds.
Have processed 11500 records in 0:00:51.159171 seconds.
Have processed 12000 records in 0:00:41.148917 seconds.
Have processed 12500 records in 0:00:55.053741 seconds.
Have processed 13000 records in 0:00:44.435864 seconds.
Have processed 13500 records in 0:00:54.325351 seconds.
Have processed 14000 records in 0:00:44.520431 seconds.
Have processed 14500 records in 0:00:23.639056 seconds.
Have processed 15000 records in 0:00:11.627489 seconds.
Writing parcel datums took: 0:26:51.276418 seconds.


C:\Users\python-dev\AppData\Local\Temp\ipykernel_6232\3081351703.py:53: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  records_df = pd.read_csv("parcel_records.csv")


,parcel_number,year,parcel_address,owner,owner_address,sale_status,property_class,tax_status,net_taxable,tax_rate,total_tax,township,acreage,homesite_val,dwelling_val,dept_rev_val,total_billed,total_unpaid
count,6.734900e+04,67349.000000,67348,67349,67349,5255,67349.000000,67349,67349,67349.000000,67349,67349,67349.000000,6.734900e+04,6.734900e+04,6.734900e+04,67349,67349
unique,NaN,NaN,52034,36468,36010,8,NaN,4,29230,NaN,34750,4,NaN,NaN,NaN,NaN,37909,30
top,NaN,NaN,"N 64TH ST EAST SAINT LOUIS, IL 62204",ST CLAIR COUNTY TRUSTEE,"10 PUBLIC SQ BELLEVILLE, IL, 62220",TRUSTSUB,NaN,Taxable,0,NaN,0.00,EAST ST LOUIS,NaN,NaN,NaN,NaN,0.00,0.00
freq,NaN,NaN,94,2435,3385,1984,NaN,63169,19830,NaN,19830,24722,NaN,NaN,NaN,NaN,19856,65508
mean,4.275571e+09,2021.056096,NaN,NaN,NaN,NaN,85.111865,NaN,NaN,9.650228,NaN,NaN,0.925005,5.601121e+03,2.211663e+04,2.782417e+04,NaN,NaN
std,2.748777e+09,3.311405,NaN,NaN,NaN,NaN,488.895593,NaN,NaN,4.911541,NaN,NaN,8.628885,3.366600e+04,7.226747e+04,9.056739e+04,NaN,NaN
min,1.110200e+09,2004.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN
25%,2.150122e+09,2022.000000,NaN,NaN,NaN,NaN,30.000000,NaN,NaN,7.793700,NaN,NaN,0.080000,4.670000e+02,0.000000e+00,6.190000e+02,NaN,NaN
50%,2.270215e+09,2022.000000,NaN,NaN,NaN,NaN,40.000000,NaN,NaN,9.571100,NaN,NaN,0.160000,1.710000e+03,6.745000e+03,8.991000e+03,NaN,NaN
75%,8.090303e+09,2022.000000,NaN,NaN,NaN,NaN,40.000000,NaN,NaN,13.834600,NaN,NaN,0.260000,5.531000e+03,2.899600e+04,3.460400e+04,NaN,NaN


,parcel,year,error
count,7853,7853.000000,7853
unique,7643,NaN,5
top,0826200002,NaN,'Billing'
freq,5,NaN,6723
mean,NaN,2012.286642,NaN
std,NaN,7.851638,NaN
min,NaN,1989.000000,NaN
25%,NaN,2005.000000,NaN
50%,NaN,2012.000000,NaN
75%,NaN,2020.000000,NaN


Proceeding to next batch...setting at index 75462..
Finished.


In [ ]:
def write_records():
    function_start = time.perf_counter()
    headers = [
        "parcel_number", "year", "parcel_address", "owner", "owner_address", 
        "sale_status", "property_class", "tax_status", "net_taxable", 
        "tax_rate", "total_tax", "township", "acreage", "homesite_val", 
        "dwelling_val", "dept_rev_val", "total_billed", "total_unpaid"
    ]
    parcel_records = []
    records_missed = []
    info_missing_ct = 0
    hundred_loop_start = time.perf_counter()

    for index, row in parcel_list.iterrows():
        if index == 0:
            continue
        else:
            listing_year = row['Year']
            listing_number = row['Property Account Number']
            try:
                parcel_info = p.submit(scrape_parcel_pg(listing_number, listing_year))
            except (TypeError, ValueError, IndexError, KeyError) as err:
                info_missing_ct += 1
                records_missed.append([listing_number, listing_year, err])
            else:
                parcel_records.append(list(parcel_info.values()))
        if index % 20 == 0:
            for_write = pd.DataFrame(parcel_records, columns=headers)
            if index == 20:
                for_write.to_csv("parcel_records.csv", index=False, mode="a")
            else:
                for_write.to_csv("parcel_records.csv", index=False, header=False, mode="a")
            if index % 100 == 0:
                hundred_loops_t = timedelta(seconds=time.perf_counter()-hundred_loop_start)
                print(f"At {index}: missed {info_missing_ct} over {hundred_loops_t}")
                missed_write = pd.DataFrame(records_missed, columns=["parcel", "year", "error"])
                if index == 100:
                    missed_write.to_csv("missed_parcels.csv", index=False, mode="a")
                else:
                    missed_write.to_csv("missed_parcels.csv", index=False, header=False, mode="a")
                records_missed.clear()
                hundred_loop_start = time.perf_counter()
            parcel_records.clear()
    write_duration = timedelta(seconds=time.perf_counter()-function_start)
    print(f"{info_missing_ct} parcel records missing information. Duration: {write_duration}")


In [ ]:
write_records()

In [ ]:
records_df = pd.read_csv("parcel_records.csv")
display(records_df.dtypes)
display(records_df.sample(7))
display(records_df.describe(include="all"))
